## Codes

The steps to estimate the rating are:
1. Define _active user_ (subscript a) and his/her _movie_ (subscript j) to be rated.
2. Get all users' ID from the training set 
3. For each user, do:
    - Get his/her votes (set of logs)
    - If he/she voted in the _movie_ defined above, do:
        - Get Movies both users (active one and training one) voted in and calculate ```w(a,i)``` and the difference between the user vote on the iterating movie and his/her average.
4. Compute normalizing factor _k_.
5. Estimate active user rate on _movie_.
6. Return estimation

In [0]:
import numpy as np

Our dataset have the users' id in the second column and the ratings in the last one. Let's define it.

In [0]:
user_column = 1
ratings_column = 2
movie_column = 0

In [0]:
def mtxslv_user_ratings(user_id, dataset):
  """
  Receives user_id and dataset. Look for all
  occurences of user_id in dataset and returns 
  such subset.

  If no user_id is found, return an empty
  numpy array. 
  """
  subset = [] # the same thing as I_i (set of item user_id has voted)
  for it in range(0,np.shape(dataset)[0]):
    if (dataset[it,user_column] == user_id):
      subset.append(dataset[it,:].tolist())  
  return np.array(subset)

In [0]:
# Python program to illustrate the intersection 
# of two lists in most simple way 
# from https://www.geeksforgeeks.org/python-intersection-two-lists/
def intersection(lst1, lst2): 
	lst3 = [value for value in lst1 if value in lst2] 
	return lst3 

In [0]:
def mtxslv_collab_filter(instance, product_id,training_set):
  """
  Receives the instance (a set of logs), the product_id which estimation
  needs to be calculated, and the training set (a bigger set of logs). All 
  parameters must be numpy. 

  The user in instance is the active one.

  Returns the estimation of rating user in instance must give to product_id. 
  """
  training_set_list = training_set.tolist()
  average_rating_active_user = np.mean(instance[:,ratings_column]) # \overine{v_a}

  w = [] # list of all weights/correlation measures
  k = 0  # normalizing term
  user_vote_minus_average = [] # v_ij - \overline{v_i}

  # let's get all user ids
  user_ids = set(training_set[:,user_column])

  # now iterate each, looking for those that haven't rated product_id
  for user in user_ids:
    user_subset = mtxslv_user_ratings(user,training_set)
    #print("user ", user)
    #print(user_subset)
    if(user_subset[:,movie_column].tolist().count(product_id)): # has user voted product_id? If yes
      #print("user ",user," has voted in product ", product_id)
      user_average = np.mean(user_subset[:,ratings_column])   # get the average of this user ratings
      #print("user ",user,"has average ",user_average)
      movies_both_users_voted = intersection(user_subset[:,movie_column].tolist(),
                                             instance[:,movie_column].tolist()) # and calculate the movies this user and the active user rated
      #print("movies both users voted:",movies_both_users_voted)
      user_index_product_id = user_subset[:,movie_column].tolist().index(product_id) # Calculate what index for user
                                                                                     # is related to product_id 
      
      w_numerator = 0                             # Numerator for w formula: \sum_{}^{}\mathop{}_{\mkern-5mu j} (v_{a,j} - \overline{v_a} )(v_{i,j} - \overline{v_i})
      w_active_denominator_factor = 0             # Active user factor for w denominator: \sum_{}^{}\mathop{}_{\mkern-5mu j} (v_{a,j} - \overline{v_a} )^2
      w_training_usr_denominator_factor = 0       # Training user factor for w denominator: \sum_{}^{}\mathop{}_{\mkern-5mu j} (v_{i,j} - \overline{v_i} )^2

      for movie in movies_both_users_voted: # Now, iterate over the movies both users rated 
        
        training_user_index_for_movie = user_subset[:,movie_column].tolist().index(movie) # index of user vote for movie
        active_user_index_for_movie = instance[:,movie_column].tolist().index(movie)      # index of active user vote for movie

        w_numerator = w_numerator + (instance[active_user_index_for_movie,ratings_column]-average_rating_active_user)*(user_subset[training_user_index_for_movie,ratings_column]-user_average)
        w_active_denominator_factor = w_active_denominator_factor + np.power((instance[active_user_index_for_movie,ratings_column]-average_rating_active_user),2)
        w_training_usr_denominator_factor = w_training_usr_denominator_factor + np.power( (user_subset[training_user_index_for_movie,ratings_column]-user_average) ,2)

      w.append(w_numerator/np.sqrt(w_active_denominator_factor*w_training_usr_denominator_factor))
      user_vote_minus_average.append(user_subset[user_index_product_id,ratings_column]-user_average)

  #print(w)    
  k = 1 / np.sum( np.abs(w) )
  #print(k)
  #print(user_vote_minus_average)
  estimation =  average_rating_active_user + k* np.dot(w,user_vote_minus_average) 

  return estimation


## REFERENCES:

* https://stackoverflow.com/questions/21860605/python-remove-lists-from-list-of-lists-similar-functionality-to-pop

* https://www.geeksforgeeks.org/python-intersection-two-lists/

* https://www.programiz.com/python-programming/methods/list/index